In [1]:
import torch
from transformers import T5ForConditionalGeneration, T5Tokenizer

# base = "cointegrated/rut5-base-multitask"
tokenizer = T5Tokenizer.from_pretrained('epoch_7/model_36/T5_para_tok/')
model = T5ForConditionalGeneration.from_pretrained('epoch_7/model_36/T5_para')
if torch.cuda.is_available():
    model.cuda()

def generate(text, **kwargs):
    inputs = tokenizer(text, return_tensors='pt').to(model.device)
    with torch.no_grad():
        hypotheses = model.generate(**inputs, num_beams=5, **kwargs)
    return tokenizer.decode(hypotheses[0], skip_special_tokens=True)


In [2]:
from transformers import (
    AdamW,
    T5ForConditionalGeneration,
    T5Tokenizer,
    get_linear_schedule_with_warmup
)
import torch
from datasets import load_dataset

In [ ]:
import pandas as pd
import os

dfs = {file.strip('.tsv'): pd.read_csv(f'../data/clean/{file}', sep='\t') for file in os.listdir('../data/clean/')}

In [ ]:
m = pymorphy2.MorphAnalyzer()

In [ ]:
from razdel import tokenize

def jaccard(list1, list2):
    intersection = len(list(set(list1).intersection(list2)))
    union = (len(list1) + len(list2)) - intersection
    return float(intersection) / union

def lemma_jaccard(s1, s2):
    s1 = [m.parse(_.text)[0].normal_form for _ in tokenize(s1) if _.text.isalpha()]
    s2 = [m.parse(_.text)[0].normal_form for _ in tokenize(s2) if _.text.isalpha()]
#     print(s1, s2)
    return jaccard(s1, s2)

In [ ]:
from razdel import sentenize

In [ ]:
# .str.replace("«", '"').str.replace('»', '"')
# .str.replace('\xa0', ' ')
# .str.replace('ё', 'е')
# .str.replace('—', '-') 
# .str.strip()

for k in dfs:
    j = []
    for t in ['text1', 'text2']:
        dfs[k][t] = dfs[k][t].str.replace('«', '"').str.replace('»', '"')
        dfs[k][t] = dfs[k][t].str.replace('\xa0', ' ').str.strip()
        dfs[k][t] = dfs[k][t].str.replace('ё', 'е')
        dfs[k][t] = dfs[k][t].str.replace('—', '-') 
    for i, row in tqdm.tqdm(dfs[k].iterrows(), total=len(dfs[k])):
        j.append(lemma_jaccard(row['text1'], row['text2']))
    dfs[k]['jaccard'] = j
    dfs[k]['sents_equal'] = dfs[k].apply(lambda x: len(list(sentenize(x['text1']))) == \
                                                   len(list(sentenize(x['text2']))), axis=1)

In [ ]:
dfs.keys()

In [ ]:
# tapaco.to_csv('../data/clean/tapaco_clean.tsv', sep='\t', index=False)

In [ ]:
tapaco = dfs['apaco_cleano_clean']
tapaco = tapaco[(tapaco.jaccard < 0.6) | ((tapaco.jaccard > 0.6) & (tapaco.sents_equal == False))]
tapaco = tapaco.drop_duplicates('cluster_id')

In [ ]:
# paraplag.to_csv('../data/clean/paraplag_clean.tsv', sep='\t', index=False)

In [ ]:
paraplag = dfs['paraplag_clean']
paraplag = paraplag[(paraplag.jaccard < 0.6) | ((paraplag.jaccard > 0.6) & (paraplag.sents_equal == False))]
paraplag = paraplag.drop('paraplag_clean', axis=1)

In [ ]:
ppp = dfs['ppp_clean']

In [ ]:
# ppp.to_csv('../data/clean/ppp_clean.tsv', sep='\t', index=False)

In [ ]:
ppp = ppp.drop('ppp_clean', axis=1)
ppp = ppp[ppp.jaccard < 0.6]
ppp = ppp.drop_duplicates('text1').drop_duplicates('text2')

In [ ]:
rltc = dfs['rltc_clean']
rltc = rltc.drop('rltc_clean', axis=1)
# rltc.to_csv('../data/clean/rltc_clean.tsv', sep='\t', index=False)

In [ ]:
rltc = rltc[(rltc.jaccard < 0.6) | ((rltc.jaccard > 0.6) & (rltc.sents_equal == False))]
rltc = rltc.drop_duplicates('text1').drop_duplicates('text2')

In [ ]:
leipzig = dfs['leipzig_clean'].drop('leipzig_clean', axis=1)
# leipzig.to_csv('../data/clean/leipzig_clean.tsv', sep='\t', index=False)

In [ ]:
leipzig = leipzig[(leipzig.jaccard < 0.6) | ((leipzig.jaccard > 0.6) & (leipzig.sents_equal == False))]
leipzig = leipzig.drop_duplicates('text1').drop_duplicates('text2')

In [ ]:
rysshe = dfs['rysshe_clean'].drop('rysshe_clean', axis=1)
# rysshe.to_csv('../data/clean/rysshe_clean.tsv', sep='\t', index=False)
rysshe = rysshe[(rysshe.jaccard < 0.6) | ((rysshe.jaccard > 0.6) & (rysshe.sents_equal == False))]
rysshe = rysshe.drop_duplicates('text1').drop_duplicates('text2')

In [ ]:
mcoco = dfs['mcoco_clean']

In [ ]:
mcoco = mcoco.drop('mcoco_clean', axis=1)
# mcoco.to_csv('../data/clean/mcoco_clean.tsv', sep='\t', index=False)
mcoco = mcoco[(mcoco.jaccard < 0.6) | ((mcoco.jaccard > 0.6) & (mcoco.sents_equal == False))]
mcoco = mcoco[(mcoco['ent1'] * mcoco['ent2']) > 0.85]

In [ ]:
opus = dfs['opusparcus_clean'].drop('opusparcus_clean', axis=1)

In [ ]:
opus.to_csv('../data/clean/opusparcus_clean.tsv', sep='\t', index=False)

In [ ]:
opus = opus[opus['jaccard'] < 0.6]
opus = opus.drop_duplicates('text1').drop_duplicates('text2')
opus['ratio_toks'] = opus.apply(lambda x: len(list(tokenize(x['text1']))) / len(list(tokenize(x['text2']))), axis=1)
opus = opus[(opus['ratio_toks'] < 2) & (opus['ratio_toks'] > 0.4)]

In [ ]:
opus.head(2)

In [ ]:
a = rysshe.sample(1).iloc[0]
print(a['text1'])
print(a['text2'])

In [ ]:
data = pd.concat([d[['text1', 'text2', 'corpus']] for d in [opus, ppp, paraplag, mcoco, leipzig ]])

In [ ]:
data = data.reset_index(drop='index')

In [ ]:
data.to_csv('../data/clean/combined_filtered.tsv', sep='\t', index=False)

In [4]:
import pandas as pd

In [5]:
df = pd.read_csv('../data/clean/combined_filtered.tsv', sep='\t')

In [6]:
df = df[['text1', 'text2']]

In [7]:
import numpy as np

In [8]:
import tqdm

data = []

for texts in tqdm.tqdm(df.sample(len(df)).values):
    np.random.shuffle(texts)
    data.append((f'paraphrase | {texts[0]}', texts[1]))

100%|██████████| 665453/665453 [00:01<00:00, 485688.21it/s]


In [9]:
data = list(set(data))

In [10]:
len(data)

665439

In [12]:
1e-4 / 2

5e-05

In [13]:
optimizer = torch.optim.Adam(params = [p for p in model.parameters() if p.requires_grad],
                             lr=6e-05)

In [ ]:
def predict(x, n=3):
    inputs = tokenizer(x, return_tensors='pt')
    inputs = {k: v.to(model.device) for k, v in inputs.items()}
    with torch.no_grad():
        hypotheses = model.generate(
            **inputs, 
            do_sample=True, 
            top_p=0.9, 
            num_return_sequences=3, 
            repetition_penalty=2.5,
            max_length=64,
        )
    return [tokenizer.decode(h, skip_special_tokens=True) for h in hypotheses]

In [14]:
train, val = data[:-1000], data[-1000:]

In [15]:
len(train)

664439

In [16]:
len(val)

1000

In [17]:
import gc

def cleanup():
    gc.collect()
    torch.cuda.empty_cache()

cleanup()

In [18]:
device = torch.device('cuda')
model.to(device);

In [19]:
from tqdm import tqdm_notebook

In [20]:
def model_eval(val, max_len=1024):
    model.eval()
    total = 0
    for x_text, y_text in val:
        x = tokenizer(x_text, return_tensors='pt', padding=True, truncation=True, max_length=max_len).to(device)
        y = tokenizer(y_text, return_tensors='pt', padding=True, truncation=True, max_length=max_len).to(device)

        loss = model(
            input_ids=x.input_ids,
            attention_mask=x.attention_mask,
            labels=y.input_ids,
            decoder_attention_mask=y.attention_mask,
            return_dict=True
        ).loss
#         loss.backward()
        total += loss.item()
    return round(total/len(val), 5)

# model_eval(val)

In [21]:
from collections import deque

In [26]:
np.mean(loss_accum)

1.2349031547129155

In [22]:
model.train();

batch_size = 6
max_len = 1024
epochs = 12
accumulation_steps = 64
save_steps = 30000

window = 5000


cleanup()

loss_accum = deque(maxlen=window)

cp_point = 0
i = 0
for e in range(8, epochs+1):
    np.random.shuffle(train)
    print(f'EPOCH: {e}')
    tq = tqdm_notebook(total=int(len(train) / batch_size))
    for batch_id in range(0, len(train), batch_size):
        batch = train[batch_id:batch_id+batch_size]
        
        text_x = [batch[i][0] for i in range(len(batch))]
        text_y = [batch[i][1] for i in range(len(batch))]
        
        x = tokenizer(text_x, return_tensors='pt', padding=True, truncation=True, max_length=max_len).to(device)
        y = tokenizer(text_y, return_tensors='pt', padding=True, truncation=True, max_length=max_len).to(device)
        y.input_ids[y.input_ids==0] = -100
        
        try:
            loss = model(
                input_ids=x.input_ids,
                attention_mask=x.attention_mask,
                labels=y.input_ids,
                decoder_attention_mask=y.attention_mask,
                return_dict=True
            ).loss

            loss.backward()
            loss_accum.append(loss.item())
        except RuntimeError:
            print('OOM')
            loss = None
            cleanup()
            continue

        tq.set_description(f'loss: {sum(loss_accum)/min(window, len(loss_accum))}')

        if i % accumulation_steps == 0:
            optimizer.step()
            optimizer.zero_grad()
            cleanup()

        if i % window == 0 and i > 0:
            cleanup()
            optimizer.param_groups[0]['lr'] *= 0.995

        if i % save_steps == 0 and i > 0:
            model.save_pretrained(f'epoch_{e}/model_{cp_point}/T5_para')
            tokenizer.save_pretrained(f'epoch_{e}/model_{cp_point}/T5_para_tok')
            print('saving...', i, optimizer.param_groups[0]['lr'])
            cp_point += 1

            try:
                test_loss = model_eval(val)
                print(f'Eval loss: {test_loss}')
                model.train();
            except RuntimeError:
                cleanup()
        i += 1
        tq.update()

EPOCH: 8


<ipython-input-22-5db04c98a57e>:21: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  tq = tqdm_notebook(total=int(len(train) / batch_size))


  0%|          | 0/110739 [00:00<?, ?it/s]

OOM
OOM
OOM
OOM
OOM
OOM
saving... 30000 5.822235056137594e-05
Eval loss: 1.02864
OOM
OOM


IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



...

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

IOPub message rate exceed

OOM
OOM


IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



OOM


KeyboardInterrupt: 

In [ ]:
cleanup()

In [ ]:
model.to('cpu');

In [ ]:
x.to('cpu')
y.to('cpu')

In [ ]:
cleanup()

In [ ]:
model.save_pretrained(f'epoch_{e}/model_{cp_point}/T5_para')
tokenizer.save_pretrained(f'epoch_{e}/model_{cp_point}/T5_para_tok')